# Discussion 10


## Calculator

Recall  that  the  reader  component  of  an  interpreter  parses  inputstrings and represents them as data structures in the implementing language. In thiscase, we need to represent Calculator expressions as Python objects. 

### Q1.1

```scheme
>>> Pair('+', Pair(1, Pair(2, Pair(3, Pair(4, nil)))))
(+ 1 2 3)
>>> Pair('+', Pair(1, Pair(Pair('*', Pair(2, Pair(3, nil))), nil)))
(+ 1 (* 2 3))
```
Box and pointer (using tabs):
```
+
    1
        2
            3
                nil
            
*
    1
        *
            2
                3
                    nil
            nil
```


### Q1.2

1. Python expression:
```
Pair('+', Pair(Pair('-', Pair(2, Pair(4, nil)), Pair(6, Pair(8, nil))))
```
2. The operator is `+`. If the `Pair` is bound to  `p`, I would obtain `+` by `p.first`.
3. The operands are the rest of the expression, `Pair('-', Pair(2, Pair(4, nil))`, `Pair(6, Pair(8, nil))`. To obtain the operands:
```python
operands = []
while p.rest is not nil:
    operands.append(p.rest.first)
    p = p.rest
```
To retrieve the first operand only, `p.rest.first`.


## Evaluation

The evaluation component of an interpreter determines the type of an expression and executes corresponding evaluation rules.

### Q2.1

1. For `(+ 2 4 6 8)`:
    - 6 calls to `calc_eval`: one for the whole expression and one for each element (operator and operands).
    - 1 call to `calc_apply`: sum the four summands.
2. For `(+ 2 (* 4 (- 6 8)))`:
    - 10 calls to `calc_eval`: 3 for the 3 pairs and 7 for each element.
    - 3 calls to `calc_apply`: one for each operator.

### Q2.2

1. We can handle comprison operators with the current form of `calc_eval` function because they are regular call expressions with the same procedures to handle call expressions. Now, we need to add new entries to the built-in OPERATORS in order to evaluate and apply them.
2. `and` is a special form, and does not follow `calc_eval` (shortcircuiting). To evaluate and apply it, we need to create new, special procedures.
3. Implementation:

```python
def calc_eval(exp):
    if isinstance(exp, Pair):
        if exp.first == 'and': # and expressions
            return eval_and(exp.rest)
        else:                       # Call expressions     
            return calc_apply(calc_eval(exp.first),list(exp.rest.map(calc_eval)))
        elif exp in OPERATORS:          # Names 
            return OPERATORS[exp]
        else:                           # Numbers
            return exp

# Recursive implementation
def eval_and(operands):
    if operands.rest is nil:
        return calc_eval(operands.first)
    elif calc_eval(operands.first) is False:
        return False
    else:
        return eval_and(operands.rest)

# Iterative implementation
def eval_and(operands):
    curr, val = operands, True
    while curr is not nil:
        val = calc_eval(curr.first)
        if val is False:
            return False
        curr = curr.rest
    return val
```


## Tail-Call Optimization

A **tail call** occurs when a function calls another function as its last action of the current frame. In this case, the frame is no longer needed, and we can remove it from memory. Scheme implements tail-call optimization. We say that a recursive function is **tail recursive** if all of its recursive calls are tail calls.

Tail recursive processes can use a constant amount of memory because each recursive call frame does not need to be saved.

When trying to identify whether a given function call within the body of a function is a tail call, we look for whether the call expression is in **tail context**. 

Given that each of the following expressions is the last expression in the body ofthe function, the following expressions are tail contexts:
- The second or third operand in an `if`expression.
- Any of the non-predicate sub-expressions in a `cond` expression (i.e.  the second expression of each clause).
- The last operand in an `and` or an `or` expression.
- The last operand in a `begin` expression’s body.
- The last operand in a `let` expression’s body.

### Q3.1

- `question-b`: it contains one recursive calls are in tail context (`if` case). Constant number of frames.
- `question-c`: it contains two recursive calls are in tail context (`if` case). Constant number of frames.
- `question-d`: it contains two recursive calls are in tail context (`if` case, second and third clauses). It does not have a constant number of frames, because the boolean clause is a recursive call.
- `question-e`: it contains two recursive calls are in tail context (`cond` case for the second clause and in the third clause a `begin` case too). It does not have a constant number of frames since the first recursive call (in the sencond clause of `cond`) is not in a tail context.

Note that some functions do not have a base case, so they never terminate execution.

### Q3.2

```scheme
(define (sum lst)
  (define (helper-sum result lst)
    (if (null? lst)
        result
        (helper-sum (+ result (car lst)) (cdr lst))
  )
  (helper-sum 0 lst)
)
```

### Q3.3

```scheme
(define (fib n)
    (define (fib-sofar i curr next)
        (if (= i n)
            curr
            (fib-sofar (+ i 1) next (+ curr next))
        )
    (fib-sofar 0 0 1)
)
```


## Extra Question

```scheme
(define (reverse lst)
    (define reverse-sofar lst lst-sofar)
        (if null? lst)
            lst-sofar
            (reverse-sofar (cdr lst) (cons (car lst) lst-sofar))
    )
    (reverse-sofar lst nil)
)

(define (append a b)
    (define (rev-append-tail a b)
        (if (null? a)
            b
            (rev-append-tail (cdr a) (cons (car a) b))
        )
    )
    (rev-append-tail (reverse a) b)
)

; ; This code does not handle lists with length zero! Basically I
; ; got confused with the names lst and rev-lst...
;(define (insert n lst)
;    (define (rev-insert lst rev-lst)
;        (cond 
;            ((null? lst) 
;                (reverse rev-lst)
;            ) 
;            ((> (car lst) n) 
;                (rev-insert (cdr lst) (cons (car lst) rev-lst) )
;            )
;            (else 
;                (rev-insert nil (append (lst) (cons n rev-lst))
;            )
;        )
;    )
;    (rev-insert (reverse lst) nil)
;)

(define (insert n lst)
    (define (rev-insert lst rev-lst)
        (cond 
            ((null? lst) 
                (cons n rev-lst)
            )
            ((> (car lst) n) 
                (append (reverse lst) (cons n rev-lst))
            )
            (else 
                (rev-insert (cdr lst) (cons (car lst) rev-lst))
            )
        )
    )
    (reverse (rev-insert lst nil))
)

```